<a href="https://colab.research.google.com/github/anthony-hung-do/arctic-tilt/blob/main/src/Part_1_TILT_on_FUNSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://www.kaggle.com/code/akarshu121/tilt-part-4-1-abstractive-approach-for-training?scriptVersionId=125284241" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
!pip install -qqq wandb

In [ ]:
from google.colab import userdata
import wandb

wandb_token = userdata.get('wandb_api')
wandb.login(key=wandb_token)

try:
    user = wandb.api.viewer()
    print("✅ Đã đăng nhập thành công với Weights & Biases!")
    print("Tên tài khoản:", user.get('username', 'Không xác định'))
except Exception as e:
    print("❌ Chưa đăng nhập hoặc token không hợp lệ.")
    print("Lỗi:", e)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: xuanlinh-work (xuanlinh-work-dut) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Đã đăng nhập thành công với Weights & Biases!
Tên tài khoản: xuanlinh-work


In [ ]:
!git clone https://github.com/uakarsh/TiLT-Implementation.git

Cloning into 'TiLT-Implementation'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (67/67), done.
remote: Total 83 (delta 35), reused 49 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (83/83), 346.46 KiB | 7.70 MiB/s, done.
Resolving deltas: 100% (35/35), done.


In [ ]:
!pip install -qqq datasets huggingface_hub fsspec sentencepiece pytorch_lightning seqeval evaluate transformers==4.37.2
!pip install -qqq --upgrade datasets huggingface_hub fsspec

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━

In [ ]:
from google.colab import userdata
from huggingface_hub import login
from huggingface_hub import whoami

hf_token = userdata.get('HF_TOKEN')
login(hf_token)

try:
    info = whoami()
    print("✅ Đã đăng nhập thành công với Hugging Face!")
    print("Tên tài khoản:", info["name"])
    print("Vai trò:", info.get("type", "Không xác định"))
except Exception as e:
    print("❌ Chưa đăng nhập hoặc token không hợp lệ.")
    print("Lỗi:", e)


✅ Đã đăng nhập thành công với Hugging Face!
Tên tài khoản: xlinh
Vai trò: user


In [ ]:
import sys
sys.path.append("./TiLT-Implementation/src/")

In [ ]:
import os
from transformers import AutoTokenizer, AutoConfig
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from pytorch_lightning.loggers import CSVLogger, WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint
import pytorch_lightning as pl

from dataset import ExtFUNSDDs
from torchvision import transforms
from tqdm.auto import tqdm

## Custom imports
from visual_backbone import Unet_encoder, RoIPool
from t5 import T5ForConditionalGenerationAbstractive, T5Stack
from transformers import AutoModel

## 1.1. Preparing the dataset

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

hf_ds = load_dataset("nielsr/funsd-layoutlmv3")
model_name = "t5-base"
## Visual Embedding extractor's parameters
in_channels = 3
num_pool_layers = 3
channels = 16
sampling_ratio = 2
spatial_scale = 48 / 384
output_size = (3,3)
load_weights = True
max_epochs = 50 #l 50 -> 2

## FUNSD Dataset specific
num_classes = 7

## Tokenizer's parameter
model_max_length = 512

t5_config = AutoConfig.from_pretrained(model_name)
## Adding new parameters
t5_config.update(dict(in_channels = in_channels, num_pool_layers = num_pool_layers,  channels = channels, model_max_length = model_max_length,
                      output_size = output_size, spatial_scale = spatial_scale, sampling_ratio = sampling_ratio, use_cache = False, load_weights = load_weights,
                      lr =  2e-4, num_classes = num_classes,max_epochs = max_epochs))

README.md:   0%|          | 0.00/770 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.3M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.54M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/149 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
def get_id2label_and_label2id():
    label2id = {'O': 0, 'B-HEADER': 1, 'I-HEADER': 2, 'B-QUESTION': 3, 'I-QUESTION': 4, 'B-ANSWER': 5, 'I-ANSWER': 6}
    id2label = {0: 'O', 1: 'B-HEADER', 2: 'I-HEADER', 3: 'B-QUESTION', 4: 'I-QUESTION', 5: 'B-ANSWER', 6: 'I-ANSWER'}
    return id2label, label2id

def convert_id_to_label(list_of_label):
  return [id2label[x] for x in list_of_label]

In [ ]:
# train_new_tags = list(map(lambda x : convert_id_to_label(x), hf_ds['train']['ner_tags']))
# test_new_tags = list(map(lambda x : convert_id_to_label(x), hf_ds['test']['ner_tags']))

In [ ]:
# hf_ds['train'] = hf_ds['train'].remove_columns("ner_tags").add_column("ner_tags", train_new_tags)
# hf_ds['test'] = hf_ds['test'].remove_columns("ner_tags").add_column("ner_tags", test_new_tags)

### 1.2 Writing the `collate_fn` for custom handling of the dataloader

In [ ]:
class CollateFn(object):
  def __init__(self, tokenizer):
    self.tokenizer = tokenizer

  def __call__(self, list_of_ds):
    simple_keys = ["input_ids", "attention_mask", "bboxes", "pixel_values", "labels"]
    actual_batch = {}
    for key in simple_keys:
      actual_batch[key] = torch.stack([x[key] for x in list_of_ds])


    return actual_batch

In [ ]:
# sample_batch_encoding = collate_fn([train_ds[0], train_ds[1]])
# for key in sample_batch_encoding:
#   sample_batch_encoding[key] = sample_batch_encoding[key].to(device)

## 2.1 Preparing the visual model

In [ ]:
class VisualEmbedding(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.unet_encoder = Unet_encoder(in_channels = config.in_channels, channels = config.channels, num_pool_layers = config.num_pool_layers)
    self.roi_pool = RoIPool(output_size = config.output_size, spatial_scale = config.spatial_scale)
    self.proj = nn.Linear(in_features = 128 * 3 * 3, out_features = config.d_model)
    self.config = config

  def forward(self, pixel_values, bboxes):
    image_embedding = self.unet_encoder(pixel_values)
    feature_maps_bboxes = self.roi_pool(image_embedding, bboxes).flatten(2)
    projection = self.proj(feature_maps_bboxes)
    return projection

## 2.2 Preparing the semantic model

In [ ]:
class TiLTTransformer(nn.Module):
  def __init__(self, config):
    super().__init__()
    self.config = config
    self.visual_embedding_extractor = VisualEmbedding(config)
    self.t5_model = T5ForConditionalGenerationAbstractive(config)


  def generate(self, batch):
    total_embedding = self.common_step(batch)
    return self.t5_model.generate(input_embeds = total_embedding)

  def common_step(self, batch):
    ## Visual embedding
    visual_embedding = self.visual_embedding_extractor(pixel_values = batch['pixel_values'], bboxes = batch['bboxes'])

    ## Semantic embedding from t5_model's embedding layer
    semantic_embedding = self.t5_model.shared(batch['input_ids'])

    ## Net embedding is addition of both the embeddings
    total_embedding = visual_embedding + semantic_embedding

    return total_embedding

  def forward(self, batch):

    total_embedding = self.common_step(batch)

    ## This is then fed to t5_model
    final_output = self.t5_model(attention_mask = batch['attention_mask'], inputs_embeds = total_embedding,
                            labels = batch['labels'])

    return final_output

In [ ]:
# tilt_model = TiLTTransformer(t5_config).to(device)
# output = tilt_model(sample_batch_encoding)

## 3.1 Preparing the metrics to evaluate the predictions

In [ ]:
import evaluate

def get_labels(predictions, references):

    # Transform predictions and references tensors to numpy arrays
    if predictions.device.type == "cpu":
        y_pred = predictions.detach().clone().numpy()
        y_true = references.detach().clone().numpy()

    else:
        y_pred = predictions.detach().cpu().clone().numpy()
        y_true = references.detach().cpu().clone().numpy()

    # Remove ignored index (special tokens)
    true_predictions = [
        [id2label[p] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    true_labels = [
        [id2label[l] for (p, l) in zip(pred, gold_label) if l != -100]
        for pred, gold_label in zip(y_pred, y_true)
    ]
    return true_predictions, true_labels

In [ ]:
# labels = sample_batch_encoding['labels']

In [ ]:
# true_predictions, true_labels = get_labels(predictions = output.logits.argmax(axis = -1), references = labels)

In [ ]:
# eval_metric = evaluate.load("seqeval")
# metric = eval_metric.compute(predictions = true_predictions, references = true_labels)

## Part: 4 Writing the `pytorch_lightning` code for training on FUNSD

In [ ]:
id2label, label2id = get_id2label_and_label2id()
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Lambda(lambda x : 2 * x - 1)])

## Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True, model_max_length = model_max_length)

train_ds = ExtFUNSDDs(hf_ds['train'],tokenizer = tokenizer, transform = transform)
val_ds = ExtFUNSDDs(hf_ds['test'],tokenizer = tokenizer, transform = transform)

collate_fn = CollateFn(tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
class DataModule(pl.LightningDataModule):

  def __init__(self, train_dataset, eval_dataset, batch_size: int = 2):
    super(DataModule, self).__init__()
    self.batch_size = batch_size
    self.train_dataset = train_dataset
    self.eval_dataset = eval_dataset

  def train_dataloader(self):
    return DataLoader(self.train_dataset, batch_size=self.batch_size,
                      shuffle=True, collate_fn = collate_fn)

  def val_dataloader(self):
    return DataLoader(self.eval_dataset, batch_size=self.batch_size,
                      shuffle=False, collate_fn = collate_fn)

In [ ]:
class TiltModel(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.tilt_model = TiLTTransformer(config)
        self.eval_metric = evaluate.load("seqeval")

    def forward(self, batch):
        return self.tilt_model(batch)

    def training_step(self, batch, batch_idx):
        output = self(batch)
        loss = output.loss

        predictions = output.logits.argmax(dim=-1)
        true_predictions, true_labels = get_labels(predictions = predictions, references =  batch['labels'])

        results = self.eval_metric.compute(predictions=true_predictions, references=true_labels)
        self.log("train_loss", output.loss.item(),
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_fl",
                 results["overall_f1"], prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_recall",
                 results["overall_recall"], prog_bar=True, on_epoch=True, logger=True)
        self.log("train_overall_precision",
                 results["overall_precision"], prog_bar=True, on_epoch=True, logger=True)

        return loss

    def validation_step(self, batch, batch_idx):
        output = self(batch)
        loss = output.loss

        predictions = output.logits.argmax(dim=-1)
        true_predictions, true_labels = get_labels(predictions = predictions, references =  batch['labels'])
        results = self.eval_metric.compute(predictions=true_predictions, references=true_labels)

        self.log("val_loss", output.loss.item(),
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_fl", results["overall_f1"],
                 prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_recall",
                 results["overall_recall"], prog_bar=True, on_epoch=True, logger=True)
        self.log("val_overall_precision",
                 results["overall_precision"], prog_bar=True, on_epoch=True, logger=True)

        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.config.lr)
        return optimizer

In [ ]:
def perform_evaluation(path: str=None, pl_model=None, pl_dl=None):
    print("Evaluating the model")
    if path is not None:
      pl_model = pl_model.load_from_checkpoint(path, config = t5_config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    eval_metric = evaluate.load("seqeval")
    pl_model = pl_model.to(device)
    pl_model.eval();

    for idx, batch in enumerate(tqdm(pl_dl.val_dataloader())):
      # move batch to device
      batch = {k: v.to(device) for k, v in batch.items()}

      with torch.no_grad():
        outputs = pl_model(batch)

        predictions = outputs.logits.argmax(-1)
        true_predictions, true_labels = get_labels(predictions, batch["labels"])
        eval_metric.add_batch(references=true_labels,predictions=true_predictions)

    results = eval_metric.compute()

    metrics = {}
    for key in ['overall_precision', 'overall_recall', 'overall_f1', 'overall_accuracy']:
      print_statement = '{0: <30}'.format(str(key) + " has value:")
      print(print_statement, results[key])
      metrics[key] = results[key]

    return metrics

In [ ]:
checkpoint_callback = ModelCheckpoint(dirpath="./tilt/models",monitor="val_overall_fl", mode="max", filename='tilt_best_ckpt', save_top_k = 1)
logger = CSVLogger("./tilt/logs", name="funsd_dataset")

In [ ]:
wandb.init(config=t5_config, project="TILT on FUNSD")
wandb_logger = WandbLogger(project="TILT on FUNSD", log_model = False, entity="hxlinh")

In [ ]:
trainer = pl.Trainer(default_root_dir="./tilt/logs", devices="auto",
                     accelerator="auto",max_epochs= t5_config.max_epochs,
                     logger=wandb_logger,callbacks=[checkpoint_callback],
                     log_every_n_steps = 5)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
pl_model = TiltModel(t5_config)
pl_dl = DataModule(train_ds, val_ds, batch_size = 2)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

Weights loaded successfully!


In [ ]:
trainer.fit(pl_model, pl_dl)

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name       | Type            | Params | Mode 
-------------------------------------------------------
0 | tilt_model | TiLTTransformer | 224 M  | train
-------------------------------------------------------
224 M     Trainable params
0         Non-trainable params
224 M     Total params
897.139   Total estimated model params size (MB)
607       Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training: |          | 0/? [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (626 > 512). Running this sequence through the model will result in indexing errors


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


In [ ]:
ckpt_folder = "./tilt/models"
if os.path.exists(ckpt_folder):
  ckpt_path = os.path.join(ckpt_folder, os.listdir(ckpt_folder)[0])
else:
  ckpt_path = None

In [ ]:
# metrics = perform_evaluation(path = ckpt_path, pl_model = pl_model, pl_dl = pl_dl) #l changed to TiltModel
metrics = perform_evaluation(path = ckpt_path, pl_model = TiltModel, pl_dl = pl_dl)

Evaluating the model
Weights loaded successfully!


  0%|          | 0/25 [00:00<?, ?it/s]

overall_precision has value:   0.5561210065026859
overall_recall has value:      0.4434174932371506
overall_f1 has value:          0.4934152765583846
overall_accuracy has value:    0.8273112079013557


In [ ]:
print(metrics)

{'overall_precision': np.float64(0.5561210065026859), 'overall_recall': np.float64(0.4434174932371506), 'overall_f1': np.float64(0.4934152765583846), 'overall_accuracy': 0.8273112079013557}
